In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

## **Загрузка данных**

In [ ]:
of = pd.read_csv('https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv')[['text', 'y']].replace({'y' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}})
of.columns = ['text', 'true']
of = of[of.true != 'NEUT']
of

,text,true
0,Электив сам по себе скучноватый.,NGTV
1,"Я не считаю, что полученные знания помогут мне...",NGTV
6,"Достаточно интересный электив, но на нем придё...",PSTV
8,"В общем и целом было весело, домашки немного, ...",PSTV
9,"Просто здорово, подойдет для активных и полуак...",PSTV
...,...,...
137,"Все говорит приблизительно, относительно, при ...",NGTV
138,Такая теория никак не поможет ребятам заинтере...,NGTV
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV
141,"Толку учить пользоваться программой, где самый...",NGTV


In [ ]:
models = pd.DataFrame(columns=['name', 'precision', 'recall', 'F-score [macro]', '~labels', 'time [s]', 'comment'])

In [ ]:
test = pd.DataFrame({'text': ['Я ненавижу тебя', 'Чай', '...', 'Доброго утра', 'С новым годом', 'Я очень люблю тебя']})

## **Обнимающее литсо**

In [ ]:
%%capture
!pip install transformers[sentencepiece]

In [ ]:
from transformers import pipeline, FeatureExtractionMixin

In [ ]:
model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment" # seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka blanchefort/rubert-base-cased-sentiment-rurewiews
face = pipeline('sentiment-analysis', model=model_name)

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Смотрим, что модель выводит, чтобы настроить labels

In [ ]:
# Тест вывода
results = face(test['text'].to_list())
print(results)
results = [row['label'] for row in results]
test['pred'] = results
test

[{'label': 'NEGATIVE', 'score': 0.988459587097168}, {'label': 'NEUTRAL', 'score': 0.9950726628303528}, {'label': 'NEUTRAL', 'score': 0.9954538345336914}, {'label': 'POSITIVE', 'score': 0.9523290991783142}, {'label': 'POSITIVE', 'score': 0.9842379689216614}, {'label': 'POSITIVE', 'score': 0.996340811252594}]


,text,pred
0,Я ненавижу тебя,NEGATIVE
1,Чай,NEUTRAL
2,...,NEUTRAL
3,Доброго утра,POSITIVE
4,С новым годом,POSITIVE
5,Я очень люблю тебя,POSITIVE


In [ ]:
labels = { 'NEGATIVE' : 'NGTV', 'NEUTRAL' : 'NEUT', 'POSITIVE' : 'PSTV'}
# labels = {'pos' : 'PSTV', 'neg' : 'NGTV'}
# labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV'}
# labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV', 'very_positive' : 'PSTV', 'very_negative' : 'NGTV'}
# labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV', 'skip' : 'NEUT', 'speech' : 'NEUT'}

# labels = { 'LABEL_1' : 'NGTV', 'LABEL_0' : 'NEUT', 'LABEL_2' : 'PSTV'}
# labels = { 'LABEL_0' : 'NGTV', 'LABEL_1' : 'NEUT', 'LABEL_2' : 'PSTV', 'LABEL_3' : 'NEUT', 'LABEL_4' : 'NEUT'}
# labels = { 'LABEL_1' : 'NGTV', 'LABEL_0' : 'PSTV'}
# labels = { 'LABEL_1' : 'NGTV', 'LABEL_0' : 'NEUT', 'LABEL_2' : 'PSTV', 'LABEL_3': 'NEUT', 'LABEL_4': 'NGTV'}

# labels = { 'sadness' : 'NGTV', 'fear' : 'NGTV', 'anger' : 'NGTV', 'surprise' : 'NEUT', 'no_emotion' : 'NEUT', 'joy' : 'PSTV' }
# labels = { 'sadness' : 'NGTV', 'fear' : 'NGTV', 'anger' : 'NGTV', 'neutral' : 'NEUT', 'enthusiasm' : 'PSTV', 'happiness' : 'PSTV' }

list(labels.keys())

['NEGATIVE', 'NEUTRAL', 'POSITIVE']

In [ ]:
%%time
of[model_name] = [row['label'] for row in face(of['text'].to_list())]

CPU times: user 10.8 s, sys: 33 ms, total: 10.9 s
Wall time: 13.9 s


In [ ]:
cols = list(of.columns)
a, b = cols.index(cols[-1]), cols.index(cols[2])
cols[b], cols[a] = cols[a], cols[b]
of = of[cols]

of = of.replace({model_name : labels})
of

,text,true,blanchefort/rubert-base-cased-sentiment-rusentiment
0,Электив сам по себе скучноватый.,NGTV,NGTV
1,"Я не считаю, что полученные знания помогут мне...",NGTV,NEUT
6,"Достаточно интересный электив, но на нем придё...",PSTV,NEUT
8,"В общем и целом было весело, домашки немного, ...",PSTV,PSTV
9,"Просто здорово, подойдет для активных и полуак...",PSTV,NEUT
...,...,...,...
137,"Все говорит приблизительно, относительно, при ...",NGTV,NEUT
138,Такая теория никак не поможет ребятам заинтере...,NGTV,NEUT
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV,NEUT
141,"Толку учить пользоваться программой, где самый...",NGTV,NEUT


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

def custom_f1(y_pred, y_true, y_possible=None, mode='fact', average='macro'):
  y_min = []
  y_max = []
  for i in range(len(y_pred)):
    if (not(pd.isnull(y_possible[i]))):
      if (y_pred[i] == y_true[i]):
        y_min.append(y_possible[i])
        y_max.append(y_true[i])
      else:
        y_min.append(y_true[i])
        y_max.append(y_possible[i])
    else:
      y_max.append(y_true[i])
      y_min.append(y_true[i])

  if (mode == 'fact'):
    return alotmetrics(y_true, y_pred, average=average)[:3]
  elif (mode == 'min'):
    return alotmetrics(y_min, y_pred, average=average)[:3]
  elif (mode == 'max'):
    return alotmetrics(y_max, y_pred, average=average)[:3]
  elif (mode == 'to table'):
    # fact min max
    return ("{}\t{}\t{}".format(alotmetrics(y_true, y_pred, average=average)[2], alotmetrics(y_min, y_pred, average=average)[2], alotmetrics(y_max, y_pred, average=average)[2]))
  else:
    print("Допустимые режимы: min/fact/max")
    return

In [ ]:
print(model_name)
print('Precision', 'Recall', 'F-score')
print(alotmetrics(of['true'], of[model_name], average='macro')[:3])
print(alotmetrics(of['true'], of[model_name], average='micro')[:3])
print(alotmetrics(of['true'], of[model_name], average='weighted')[:3])

blanchefort/rubert-base-cased-sentiment-rusentiment
Precision Recall F-score
(0.6551724137931035, 0.2650980392156863, 0.3632710468153506)
(0.42857142857142855, 0.42857142857142855, 0.42857142857142855)
(0.9794745484400658, 0.42857142857142855, 0.5761359396079107)


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
metrics = alotmetrics(of['true'], of[model_name], average='macro')[:3]

if (not(model_name in models['name'].unique())):
  row = {'name': model_name, 'precision' : metrics[0], 'recall' : metrics[1], 'F-score [macro]' : metrics[2], '~labels' : list(labels.keys()), 'time [s]': 18.3, 'comment': '~ 1.05 G'}
  models = models.append(row, ignore_index=True)

models

In [ ]:
of.to_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/senino/face_preds.csv")
models.to_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/senino/face_models.csv")

## **DeepPavlov**

QuickStart http://docs.deeppavlov.ai/en/master/intro/quick_start.html

Features http://docs.deeppavlov.ai/en/master/features/overview.html

https://github.com/deeppavlov/DeepPavlov/tree/0.15.0/deeppavlov/configs/classifiers

In [ ]:
%%capture
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install sentiment_yelp_multi_bert

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/deeppavlov/__main__.py", line 4, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/deeppavlov/deep.py", line 99, in main
    install_from_config(pipeline_config_path)
  File "/usr/local/lib/python3.7/dist-packages/deeppavlov/utils/pip_wrapper/pip_wrapper.py", line 56, in install_from_config
    requirements_files = get_config_requirements(config)
  File "/usr/local/lib/python3.7/dist-packages/deeppavlov/utils/pip_wrapper/pip_wrapper.py", line 43, in get_config_requirements
    config = parse_config(config)
  File "/usr/local/lib/python3.7/dist-packages/deeppavlov/core/commands/utils.py", line 121, in parse_config
    config = read_json(find_config(config))
  File "/usr/local/lib/python3.7/dist-packages/d

In [ ]:
from deeppavlov import configs

In [ ]:
from deeppavlov import build_model

In [ ]:
%%capture
model = build_model(configs.classifiers.sentiment_sst_multi_bert, download=True)

AttributeError: ignored

In [ ]:
# Тест вывода
results = model(test['text'].to_list())
print(results)
test['pred'] = results
test

In [ ]:
labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV'}
labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV', 'very_positive' : 'PSTV', 'very_negative' : 'NGTV'}
# labels = { 'negative' : 'NGTV', 'neutral' : 'NEUT', 'positive' : 'PSTV', 'skip' : 'NEUT', 'speech' : 'NEUT'}

In [ ]:
%%time
of['pred'] = [row for row in model(of['text'].to_list())]

CPU times: user 27 s, sys: 267 ms, total: 27.3 s
Wall time: 28.2 s


In [ ]:
of = of.replace({'pred' : labels})
of

In [ ]:
print('Precision', 'Recall', 'F-score')
print(alotmetrics(of['true'], of['pred'], average='macro')[:3])
print(alotmetrics(of['true'], of['pred'], average='micro')[:3])
print(alotmetrics(of['true'], of['pred'], average='weighted')[:3])

Precision Recall F-score
(0.46913580246913583, 0.34053838484546356, 0.24323624065892105)
(0.4125874125874126, 0.4125874125874126, 0.4125874125874126)
(0.4617974617974618, 0.4125874125874126, 0.2846128670870938)


In [ ]:
model(['input_text1', 'input_text2'])

## Обучение лица [not ready]

In [ ]:
checpoint = 'blanchefort/rubert-base-cased-sentiment-rurewiews'
save_directory = 'drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/AHTUNG'

print("Вставь в ls следующее: \n" + save_directory + '/' + checpoint.split('/')[1])

Вставь в ls следующее: 
drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/AHTUNG/rubert-base-cased-sentiment-rurewiews


In [ ]:
model.save_pretrained(save_directory + '/' + model_name.split('/')[1])
!ls drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/AHTUNG/rubert-base-cased-sentiment-rurewiews